In [1]:
import tensorflow as tf
tf.enable_eager_execution()
from tensorflow.keras.layers import Lambda, Input, Embedding, Dense, TimeDistributed, Dropout, BatchNormalization, Add, SpatialDropout1D
from tensorflow.keras.layers import CuDNNLSTM as LSTM

vocab_size=30000
inp=Input(batch_shape=(1,1,),dtype=tf.int64,name="input_sent")
emb_layer=Embedding(input_dim=vocab_size,output_dim=1500,embeddings_initializer=tf.keras.initializers.Constant(0.01))
emb=emb_layer(inp)
emb_do=SpatialDropout1D(0.5)(emb)
emb_drop_n=BatchNormalization()(emb_do)
rnn1=LSTM(1500,return_sequences=True,stateful=True)(emb_drop_n)
rnn1_n=BatchNormalization()(rnn1)
rnn2=LSTM(1500,return_sequences=True,stateful=True)(rnn1_n)
rnn2_n=BatchNormalization()(rnn2)
rnn3=LSTM(1500,return_sequences=True,stateful=True)(rnn2_n)
rnn3_n=BatchNormalization()(rnn3)
proj_weights=TimeDistributed(Dense(1500))(Add()([rnn1_n,rnn2_n,rnn3_n]))
proj_weights_gelu=Lambda(lambda x: tf.multiply(x,tf.nn.sigmoid(tf.scalar_mul(1.702,x))))(proj_weights)
dec=Dense(vocab_size,activation="softmax",name="decision")
dec_td=TimeDistributed(dec)(proj_weights_gelu)
mod=tf.keras.Model(inputs=[inp],outputs=[dec_td])


In [2]:
mod.load_weights("/home/ginter/rnnlm_models/epoch.2019-03-24-00-00.00020.last.rnnlm")

In [3]:
import data
dp=data.SubwordDataPipeline("spiece_vocab.sp")


In [4]:
dp.subword_model.IdToPiece(2)

'<s>'

In [5]:
def reset_model(m):
    lstms=[l for l in m.layers if isinstance(l,tf.keras.layers.CuDNNLSTM)]
    for l in lstms:
        l.reset_states()

In [6]:
import numpy as np

#winner=tf.argmax(res,axis=-1)
#print(winner[0][0].numpy())
#dp.subword_model.IdToPiece(int(winner))

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def prewarm(txt,mod,dp):
    reset_model(mod)
    ids=dp.subword_model.EncodeAsIds(txt)
    for i in ids:
        inp=np.asarray([[i]],dtype=np.int64)
        inp_t=tf.convert_to_tensor(inp,tf.int64)
        res=mod(inp_t)
        winner=tf.argmax(res,axis=-1)
        spiece_id=winner[0][0].numpy()
        sw=dp.subword_model.IdToPiece(int(spiece_id))
        print("Prewarm:",sw)
    winner=tf.argmax(res,axis=-1)
    spiece_id=winner[0][0].numpy()
    return int(spiece_id)

#prewarm_text="Meillä on siis pieni ongelma. Tai no oikeastaan aika suuri sellainen. 13-vuotias poikani haluaa nimittäin viikonloppuna mennä kaverinsa ja tämän perheen kanssa Ruotsin risteilylle. Kaveri siis itse ehdotti tätä ja poika haluaa todellakin mennä. Kaikki olisi minullekin täysin OK, jos poikani ei sattuisi olemaan"
#prewarm_text="Helsingin alla risteilee jopa yli satavuotisia vesiputkia. Ikä ei kuitenkaan yksin määritä putken kuntoa, ja siksi Helsingissäkään ei pystytä tekemään riskiarvioita vain sen perusteella"
prewarm_text="Kaupunkien tarpeet ovat Suomessa jääneet lähes tyystin sivuun. Viime vuosina on kuitenkin nähty, että Helsingissä ja sen naapureissa Espoossa ja Vantaalla vuokrat nousevat ja asumisen väljyys"
#prewarm_text="Vantaan terveysasemille hankittiin uusi puhelinpalvelu: soittajat jonottavat jopa 45 minuuttia"
#prewarm_text="Olipa kerran pieni prinsessa"
#prewarm_text="En mä vaan tiiä"
start=prewarm(prewarm_text,mod,dp)
lst=[]
for _ in range(100):
    inp=np.asarray([[start]],dtype=np.int64)
    inp_t=tf.convert_to_tensor(inp,tf.int64)
    res=mod(inp_t)
    probs=tf.squeeze(res).numpy()
    winner=sample(probs,0.4)
    #winner=np.random.choice(a=np.arange(probs.shape[0]),p=probs)
    #print("argmax:",tf.argmax(res,axis=-1))
    #print("winner:",winner)
    spiece_id=winner
    sw=dp.subword_model.IdToPiece(int(spiece_id))
    lst.append(sw)
    start=spiece_id
print("Final output:")
dp.subword_model.DecodePieces(lst)



Prewarm: gissa
Prewarm: ▁ja
Prewarm: ▁ovat
Prewarm: ▁muuttuneet
Prewarm: ▁usein
Prewarm: ▁vähä
Prewarm: ▁kokonaan
Prewarm: ▁.
Prewarm: ▁.
Prewarm: M
Prewarm: ▁vuosina
Prewarm: ▁on
Prewarm: ▁ollut
Prewarm: ▁tapahtunut
Prewarm: ▁,
Prewarm: ▁että
Prewarm: ▁kaupunki
Prewarm: ▁on
Prewarm: ▁Helsingissä
Prewarm: ▁ulkopuolella
Prewarm: issa
Prewarm: ▁on
Prewarm: ▁on
Prewarm: ▁Vantaalla
Prewarm: ▁on
Prewarm: tiin
Prewarm: ▁.
Prewarm: ▁ovat
Prewarm: ▁kustannukset
Prewarm: jen
Prewarm: yyttä
Prewarm: ▁on
Final output:


'erittäin suurta . Kaupunginvaltuusto päätti viime viikolla esittää , että kaupunginhallitus käsittelee kaupungin asunto-osakeyhtiölain muuttamista . Kaupunginhallitus esittää , että kaupunginhallitus päättäisi , että kaupunginhallitus päättää antaa valtuutuksen kaupunginhallitukselle . Valtuuston puheenjohtajaksi on valittu kansanedustaja Kimmo Sasi . Valtuuston puheenjohtajana toimii liikenne- ja kuntaministeri Henna Virkkunen . Lue lisää  ⁇  Liikenne- ja viestintäministeriö on myöntänyt eduskunnalle esityksen , jolla se ehdottaa vesihuoltolaitoksen ja vesihuoltolaitoksen yhteistyötä vesihuoltolaitoksen kanssa . Lue'